In [1]:
# import geopandas as gpd

# # Load the GeoJSON file
# geojson_path = 'train_data/challenge_4_bb.geojson'
# gdf = gpd.read_file(geojson_path)
# gdf

,geometry
0,"POLYGON ((115.18666 37.93093, 115.18666 36.670..."


In [2]:
# import pyproj

# def get_utm_zone(longitude):
#     return int((longitude + 180) / 6) + 1

# # Get the bounds of the geometry
# minx, miny, maxx, maxy = gdf.geometry.bounds.iloc[0]

# # Calculate UTM zone
# utm_zone = get_utm_zone(minx)

# # Check for a suitable projection using pyproj
# proj = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')

# # Get the corresponding EPSG code for the UTM zone using pyproj
# utm_crs = pyproj.CRS(f"+proj=utm +zone={utm_zone} +datum=WGS84")
# epsg_code = utm_crs.to_epsg()

# # Reproject the GeoDataFrame to the chosen EPSG code
# gdf = gdf.to_crs(epsg=epsg_code)
# gdf

,geometry
0,"POLYGON ((340637.814 4199701.949, 337955.761 4..."


In [3]:
# import numpy as np

# # Create a grid of points 5120m apart
# x = np.arange(gdf.total_bounds[0], gdf.total_bounds[2], 2560)
# y = np.arange(gdf.total_bounds[1], gdf.total_bounds[3], 2560)
# xx, yy = np.meshgrid(x, y)
# points = np.vstack([xx.ravel(), yy.ravel()]).T

# grid = gpd.GeoDataFrame(geometry=gpd.points_from_xy(points[:, 0], points[:, 1], crs=gdf.crs))
# grid

,geometry
0,POINT (337955.761 4058385.040)
1,POINT (340515.761 4058385.040)
2,POINT (343075.761 4058385.040)
3,POINT (345635.761 4058385.040)
4,POINT (348195.761 4058385.040)
...,...
3859,POINT (501795.761 4199185.040)
3860,POINT (504355.761 4199185.040)
3861,POINT (506915.761 4199185.040)
3862,POINT (509475.761 4199185.040)


In [12]:
# import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Point
# import pystac_client
# import stackstac
# import torch
# from torchvision import transforms as v2
# from box import Box
# import yaml
# import math
# from rasterio.enums import Resampling
# from tqdm import tqdm
# import rasterio
# import warnings
# import os
# import numpy as np
# import rioxarray  # Make sure to import rioxarray to extend xarray

# from src.model import ClayMAEModule

# # Set the environment variable for requester-pays 
# # NEED THIS LINE FOR LANDSAT
# os.environ['AWS_REQUEST_PAYER'] = 'requester'

# warnings.filterwarnings("ignore")

# STAC_API = "https://earth-search.aws.element84.com/v1"
# COLLECTION = "landsat-c2-l2"

# # Load the model and metadata
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ckpt = "https://clay-model-ckpt.s3.amazonaws.com/v0.5.7/mae_v0.5.7_epoch-13_val-loss-0.3098.ckpt"
# torch.set_default_device(device)

# torch.cuda.empty_cache()  # Clear GPU cache

# # Assuming grid is a GeoDataFrame with the points
# points = grid.to_crs("EPSG:4326").geometry.apply(lambda x: (x.x, x.y)).tolist()

# model = ClayMAEModule.load_from_checkpoint(
#     ckpt, metadata_path="configs/metadata.yaml", shuffle=False, mask_ratio=0
# )
# model.eval()
# model = model.to(device)

# metadata = Box(yaml.safe_load(open("configs/metadata.yaml")))

# # Function to normalize timestamp
# def normalize_timestamp(date):
#     week = date.isocalendar().week * 2 * np.pi / 52
#     hour = date.hour * 2 * np.pi / 24
#     return (math.sin(week), math.cos(week)), (math.sin(hour), math.cos(hour))

# # Function to normalize lat/lon
# def normalize_latlon(lat, lon):
#     lat = lat * np.pi / 180
#     lon = lon * np.pi / 180
#     return (math.sin(lat), math.cos(lat)), (math.sin(lon), math.cos(lon))

# def to_device(data, device):
#     if isinstance(data, torch.Tensor):
#         return data.to(device)
#     elif isinstance(data, dict):
#         return {k: to_device(v, device) for k, v in data.items()}
#     elif isinstance(data, list):
#         return [to_device(v, device) for v in data]
#     return data

# def process_point(lon, lat, model, metadata, year, device, j):
#     model.to(device)  # Ensure the model is on the correct device
#     catalog = pystac_client.Client.open(STAC_API)
#     search = catalog.search(
#         collections=[COLLECTION],
#         datetime=f"{year}-01-01/{year}-12-31",
#         bbox=(lon - 1e-5, lat - 1e-5, lon + 1e-5, lat + 1e-5),
#         max_items=10,
#         query={"eo:cloud_cover": {"lt": 80}},
#     )

#     all_items = search.get_all_items()
#     items = list(all_items)
#     if not items:
#         return None
    
#     items = sorted(items, key=lambda x: x.properties.get('eo:cloud_cover', float('inf')))
#     lowest_cloud_item = items[0]

#     epsg = lowest_cloud_item.properties["proj:epsg"]

#     poidf = gpd.GeoDataFrame(
#         pd.DataFrame(),
#         crs="EPSG:4326",
#         geometry=[Point(lon, lat)],
#     ).to_crs(epsg)

#     coords = poidf.iloc[0].geometry.coords[0]

#     size = 256
#     gsd = 30
#     bounds = (
#         coords[0] - (size * gsd) // 2,
#         coords[1] - (size * gsd) // 2,
#         coords[0] + (size * gsd) // 2,
#         coords[1] + (size * gsd) // 2,
#     )

#     stack = stackstac.stack(
#         lowest_cloud_item,
#         bounds=bounds,
#         snap_bounds=False,
#         epsg=epsg,
#         resolution=gsd,
#         dtype="float32",
#         rescale=False,
#         fill_value=0,
#         assets=["blue", "green", "red", "nir"],
#         resampling=Resampling.nearest,
#     )

#     stack = stack.compute()

#     items = []
#     dates = []
#     for item in all_items:
#         if item.datetime.date() not in dates:
#             items.append(item)
#             dates.append(item.datetime.date())

#     platform = "landsat-c2l1"
#     mean = []
#     std = []
#     waves = []
#     for band in stack.band:
#         mean.append(metadata[platform].bands.mean[str(band.values)])
#         std.append(metadata[platform].bands.std[str(band.values)])
#         waves.append(metadata[platform].bands.wavelength[str(band.values)])

#     transform = v2.Compose([v2.Normalize(mean=mean, std=std)])

#     datetimes = stack.time.values.astype("datetime64[s]").tolist()
#     times = [normalize_timestamp(dat) for dat in datetimes]
#     week_norm = [dat[0] for dat in times]
#     hour_norm = [dat[1] for dat in times]

#     latlons = [normalize_latlon(lat, lon)] * len(times)
#     lat_norm = [dat[0] for dat in latlons]
#     lon_norm = [dat[1] for dat in latlons]

#     pixels = torch.from_numpy(stack.data.astype(np.float32)).to(device)
#     pixels = transform(pixels)

#     batch_size = 16
#     num_batches = math.ceil(len(stack) / batch_size)
    
#     embeddings_list = []
#     for i in range(num_batches):
#         start_idx = i * batch_size
#         end_idx = min((i + 1) * batch_size, len(stack))
        
#         batch_pixels = pixels[start_idx:end_idx].to(device)
#         batch_time = torch.tensor(np.hstack((week_norm, hour_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
#         batch_latlon = torch.tensor(np.hstack((lat_norm, lon_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
        
#         batch_datacube = {
#             "platform": platform,
#             "time": batch_time,
#             "latlon": batch_latlon,
#             "pixels": batch_pixels,
#             "gsd": torch.tensor(stack.gsd.values).to(device),
#             "waves": torch.tensor(waves).to(device),
#         }

#         batch_datacube = to_device(batch_datacube, device)

#         try:
#             model = model.to(device)

#             with torch.no_grad():
#                 unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
#             batch_embeddings = unmsk_patch[:, 0, :].cpu().numpy()
#             embeddings_list.append(batch_embeddings)
#         except RuntimeError as e:
#             if "out of memory" in str(e):
#                 print(f"GPU OOM for point ({lon}, {lat}), batch {i+1}/{num_batches}. Trying CPU...")
#                 device = torch.device("cpu")
#                 batch_datacube = to_device(batch_datacube, device)
#                 model = model.to(device)
#                 with torch.no_grad():
#                     unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
#                 batch_embeddings = unmsk_patch[:, 0, :].numpy()
#                 embeddings_list.append(batch_embeddings)
#                 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#             else:
#                 raise e

#     embeddings = np.concatenate(embeddings_list, axis=0)
#     return embeddings

# # Specify the years for the datetime range in the search
# year = 2015

# # Initialize an empty dictionary to store results for both years
# results_dict = {"lon": [], "lat": [], "embeddings": []}

# # Iterate through the points and process each one for both years
# for i, point in enumerate(tqdm(points)):
#     lon, lat = point
#     results_dict["lon"].append(lon)
#     results_dict["lat"].append(lat)
    
#     embeddings = process_point(lon, lat, model, metadata, year, device, i)
#     if embeddings is not None:
#         results_dict["embeddings"].append(embeddings)
#     else:
#         results_dict["embeddings"].append(None)

# # Create a DataFrame from the results
# df = pd.DataFrame(results_dict)

# # Convert to a GeoDataFrame
# gdf_results = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# # Output the resulting GeoDataFrame
# gdf_results.head()


100%|██████████| 3864/3864 [53:44<00:00,  1.20it/s] 


,lon,lat,embeddings,geometry
0,115.186981,36.657277,"[[0.011127532, -0.12299517, -0.13298486, 0.186...",POINT (115.18698 36.65728)
1,115.215611,36.657709,"[[0.0035498408, -0.11508457, -0.13443036, 0.17...",POINT (115.21561 36.65771)
2,115.244241,36.658135,"[[0.005627362, -0.12565899, -0.14232168, 0.184...",POINT (115.24424 36.65814)
3,115.272872,36.658554,"[[0.019624209, -0.11788666, -0.14006627, 0.160...",POINT (115.27287 36.65855)
4,115.301503,36.658966,"[[0.030935973, -0.12229435, -0.13792522, 0.161...",POINT (115.30150 36.65897)


In [13]:
# gdf_copy = gdf_results.copy()
# gdf_copy["embeddings"] = [embedding.flatten() if embedding is not None and embedding.size > 0 else None for embedding in gdf_results["embeddings"]]

# gdf_copy.to_parquet("train_data/challenge_4.parquet")

#### Join grain/maize data

In [60]:
import geopandas as gpd
import rioxarray
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from shapely.geometry import Point

# Load your GeoDataFrame
gdf = gpd.read_parquet("train_data/challenge_4.parquet")

# Load the TIFF files as xarray Datasets
raster1_path = 'train_data/MaizeYield2015.tif'
raster2_path = 'train_data/WheatYield2015.tif'

# Convert the TIFFs to xarray datasets
raster1_xr = rioxarray.open_rasterio(raster1_path)
raster2_xr = rioxarray.open_rasterio(raster2_path)
raster1_xr = raster1_xr.rio.reproject(4326)
raster2_xr = raster2_xr.rio.reproject(4326)

# Create the geometry column using Point
gdf['geometry'] = gdf.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Function to get raster value at geometry
def get_raster_value_at_geometry_xr(geometry, raster_xr):
    try:
        clipped = raster_xr.rio.clip([geometry], raster_xr.rio.crs)
        return float(clipped.sum().values)
    except rioxarray.exceptions.NoDataInBounds as e:
        return np.nan

def process_row(geom, raster_xr):
    return get_raster_value_at_geometry_xr(geom, raster_xr)

# Parallel processing with joblib
n_jobs = 12 # Use all available cores

# Use tqdm with joblib for progress bar
raster_values_1 = Parallel(n_jobs=n_jobs)(
    delayed(process_row)(geom, raster1_xr) for geom in tqdm(gdf['geometry'], desc="Processing Raster 1"))

raster_values_2 = Parallel(n_jobs=n_jobs)(
    delayed(process_row)(geom, raster2_xr) for geom in tqdm(gdf['geometry'], desc="Processing Raster 2"))

# Assign the values back to the GeoDataFrame
gdf['maize'] = raster_values_1
gdf['wheat'] = raster_values_2

# Set values greater than 1 to NaN for raster_value_1
gdf.loc[np.abs(gdf['maize']) > 1e37, 'maize'] = np.nan

# Set values greater than 1 to NaN for raster_value_2
gdf.loc[np.abs(gdf['wheat']) > 1e37, 'wheat'] = np.nan

# Save the result to a parquet file
gdf.to_parquet("train_data/challenge_4.parquet")


Processing Raster 2: 100%|██████████| 3864/3864 [00:41<00:00, 92.87it/s]


In [64]:
import numpy as np
import tensorflow as tf
import optuna
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

class NeuralNetwork:
    def __init__(self, input_shape, layers, dropout_rate, learning_rate, device):
        self.input_shape = input_shape
        self.layers = layers
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.device = device
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Input(shape=(self.input_shape,)))
        for layer_size in self.layers:
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(self.dropout_rate))
        model.add(Dense(2, activation='linear'))  # Output layer for predicting 'maize' and 'wheat'
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])  # Use MSE for regression

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        with tf.device(self.device):
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def evaluate_model(self, X_test, y_test):
        with tf.device(self.device):
            loss, mae = self.model.evaluate(X_test, y_test)
        return loss, mae

    def predict(self, X_test):
        with tf.device(self.device):
            predictions = self.model.predict(X_test)
        return predictions

    def calculate_rmse(self, y_test, predictions):
        return np.sqrt(mean_squared_error(y_test, predictions, multioutput='raw_values'))
    
    def save_model(self, filename):
        self.model.save(filename)

    @classmethod
    def load_model(cls, filename, input_shape, device):
        loaded_model = tf.keras.models.load_model(filename)
        nn = cls(input_shape, [], 0, 0, device)  # Dummy values for layers, dropout_rate, and learning_rate
        nn.model = loaded_model
        return nn
    
def objective(trial):
    layers = []
    for i in range(trial.suggest_int('n_layers', 1, 3)):
        layers.append(trial.suggest_int(f'n_units_l{i}', 64, 512))
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    
    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
    nn.compile_model()
    
    nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    
    predictions = nn.predict(X_test)
    rmse = nn.calculate_rmse(y_test, predictions)
    
    return np.mean(rmse)

# Assuming combined_gdf is already loaded
combined_gdf = gdf.copy()
combined_gdf_filtered = combined_gdf.dropna(subset=['embeddings'])

# Set NaN values in 'maize' and 'wheat' to 0
combined_gdf_filtered['maize'] = combined_gdf_filtered['maize'].fillna(0)
combined_gdf_filtered['wheat'] = combined_gdf_filtered['wheat'].fillna(0)

# Flatten the nested arrays
def flatten_embeddings(embedding):
    if isinstance(embedding, np.ndarray) and embedding.ndim > 0:
        return np.concatenate(embedding) if embedding.ndim > 1 else embedding.flatten()
    else:
        return np.array([])  # Handle cases where embedding might be zero-dimensional or empty

X = np.array([flatten_embeddings(x) for x in combined_gdf_filtered['embeddings'] if flatten_embeddings(x).size > 0])
y = combined_gdf_filtered[['maize', 'wheat']].to_numpy()

# Ensure that X and y have matching lengths after filtering
X = np.array([x for i, x in enumerate(X) if flatten_embeddings(combined_gdf_filtered['embeddings'].iloc[i]).size > 0])
y = y[:X.shape[0]]  # Adjust y to have the same number of samples as X

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

# Optimize the hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# Print the best hyperparameters
print(study.best_params)

# Example usage with the best hyperparameters
best_params = study.best_params
layers = [best_params[f'n_units_l{i}'] for i in range(best_params['n_layers'])]
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']

nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
nn.compile_model()
history = nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
loss, mae = nn.evaluate_model(X_test, y_test)
print(f'Test MAE: {mae:.4f}')
predictions = nn.predict(X_test)
rmse = nn.calculate_rmse(y_test, predictions)
print(f'Test RMSE: {rmse}')

# Save the model
nn.save_model('models/task_4_model.h5')

# Save scaler
joblib.dump(scaler, 'models/4_scaler.joblib')


I0000 00:00:1723904598.498452    4063 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
[I 2024-08-17 14:23:18,521] A new study created in memory with name: no-name-d23c7732-6d64-4e6c-95bd-9c77daf1b7fd
I0000 00:00:1723904598.517595    4063 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1723904598.519046    4063 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-

Epoch 1/20


I0000 00:00:1723904599.667561   13952 service.cc:146] XLA service 0x7fd9f400dc80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723904599.667599   13952 service.cc:154]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2024-08-17 14:23:19.769696: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-17 14:23:20.003010: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8905


51/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13395885.0000 - mae: 2847.1567 

I0000 00:00:1723904601.482028   13952 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 14767067.0000 - mae: 2885.5044 - val_loss: 8981394.0000 - val_mae: 2823.0242
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8905823.0000 - mae: 2776.3018 - val_loss: 8773018.0000 - val_mae: 2802.1782
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8691644.0000 - mae: 2736.0481 - val_loss: 8627619.0000 - val_mae: 2788.2649
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8521707.0000 - mae: 2717.3708 - val_loss: 8583093.0000 - val_mae: 2757.5369
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8443622.0000 - mae: 2695.4302 - val_loss: 8542317.0000 - val_mae: 2730.2083
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8561479.0000 - mae: 2677.5017 - val_loss: 8548053.0000 - val_mae: 2720.4265
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8535877.0000 - mae: 2673.1191 - val_loss: 8517037.0000 - val_mae: 2705.0232
Epoch 8/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 85

[I 2024-08-17 14:24:14,076] Trial 4 finished with value: 2852.5199810506183 and parameters: {'n_layers': 2, 'n_units_l0': 198, 'n_units_l1': 85, 'dropout_rate': 0.4108308923556877, 'learning_rate': 0.0033137192398227165}. Best is trial 0 with value: 2837.744908981487.


{'n_layers': 2, 'n_units_l0': 463, 'n_units_l1': 77, 'dropout_rate': 0.3893498596969507, 'learning_rate': 0.005693862022830421}
Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 12516891.0000 - mae: 2865.1230 - val_loss: 9052540.0000 - val_mae: 2825.2517
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8771631.0000 - mae: 2763.2871 - val_loss: 8896587.0000 - val_mae: 2752.4192
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8817680.0000 - mae: 2740.2012 - val_loss: 8732529.0000 - val_mae: 2744.5759
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8530377.0000 - mae: 2676.2192 - val_loss: 8417034.0000 - val_mae: 2699.5869
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8604024.0000 - mae: 2697.1816 - val_loss: 8566418.0000 - val_mae: 2695.3264
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8401383.0000 - mae: 2628.4185 - val_loss: 8428404.0000 - val_mae: 2718.2075
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8612652.

Test RMSE: [3099.1521812  2584.47239963]


['models/4_scaler.joblib']

In [65]:
import geopandas as gpd

# Load the GeoJSON file
geojson_path = 'test_data/challenge_4_bb.geojson'
gdf = gpd.read_file(geojson_path)
gdf

,geometry
0,"POLYGON ((108.53924 34.58379, 108.53924 34.278..."


In [66]:
import pyproj

def get_utm_zone(longitude):
    return int((longitude + 180) / 6) + 1

# Get the bounds of the geometry
minx, miny, maxx, maxy = gdf.geometry.bounds.iloc[0]

# Calculate UTM zone
utm_zone = get_utm_zone(minx)

# Check for a suitable projection using pyproj
proj = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')

# Get the corresponding EPSG code for the UTM zone using pyproj
utm_crs = pyproj.CRS(f"+proj=utm +zone={utm_zone} +datum=WGS84")
epsg_code = utm_crs.to_epsg()

# Reproject the GeoDataFrame to the chosen EPSG code
gdf = gdf.to_crs(epsg=epsg_code)
gdf

,geometry
0,"POLYGON ((274295.683 3829641.202, 273472.551 3..."


In [67]:
import numpy as np

# Create a grid of points 5120m apart
x = np.arange(gdf.total_bounds[0], gdf.total_bounds[2], 2560)
y = np.arange(gdf.total_bounds[1], gdf.total_bounds[3], 2560)
xx, yy = np.meshgrid(x, y)
points = np.vstack([xx.ravel(), yy.ravel()]).T

grid = gpd.GeoDataFrame(geometry=gpd.points_from_xy(points[:, 0], points[:, 1], crs=gdf.crs))
grid

,geometry
0,POINT (273472.551 3794871.701)
1,POINT (276032.551 3794871.701)
2,POINT (278592.551 3794871.701)
3,POINT (281152.551 3794871.701)
4,POINT (283712.551 3794871.701)
...,...
233,POINT (304192.551 3828151.701)
234,POINT (306752.551 3828151.701)
235,POINT (309312.551 3828151.701)
236,POINT (311872.551 3828151.701)


In [68]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pystac_client
import stackstac
import torch
from torchvision import transforms as v2
from box import Box
import yaml
import math
from rasterio.enums import Resampling
from tqdm import tqdm
import rasterio
import warnings
import os
import numpy as np
import rioxarray  # Make sure to import rioxarray to extend xarray

from src.model import ClayMAEModule

# Set the environment variable for requester-pays 
# NEED THIS LINE FOR LANDSAT
os.environ['AWS_REQUEST_PAYER'] = 'requester'

warnings.filterwarnings("ignore")

STAC_API = "https://earth-search.aws.element84.com/v1"
COLLECTION = "landsat-c2-l2"

# Load the model and metadata
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ckpt = "https://clay-model-ckpt.s3.amazonaws.com/v0.5.7/mae_v0.5.7_epoch-13_val-loss-0.3098.ckpt"
torch.set_default_device(device)

torch.cuda.empty_cache()  # Clear GPU cache

# Assuming grid is a GeoDataFrame with the points
points = grid.to_crs("EPSG:4326").geometry.apply(lambda x: (x.x, x.y)).tolist()

model = ClayMAEModule.load_from_checkpoint(
    ckpt, metadata_path="configs/metadata.yaml", shuffle=False, mask_ratio=0
)
model.eval()
model = model.to(device)

metadata = Box(yaml.safe_load(open("configs/metadata.yaml")))

# Function to normalize timestamp
def normalize_timestamp(date):
    week = date.isocalendar().week * 2 * np.pi / 52
    hour = date.hour * 2 * np.pi / 24
    return (math.sin(week), math.cos(week)), (math.sin(hour), math.cos(hour))

# Function to normalize lat/lon
def normalize_latlon(lat, lon):
    lat = lat * np.pi / 180
    lon = lon * np.pi / 180
    return (math.sin(lat), math.cos(lat)), (math.sin(lon), math.cos(lon))

def to_device(data, device):
    if isinstance(data, torch.Tensor):
        return data.to(device)
    elif isinstance(data, dict):
        return {k: to_device(v, device) for k, v in data.items()}
    elif isinstance(data, list):
        return [to_device(v, device) for v in data]
    return data

def process_point(lon, lat, model, metadata, year, device, j):
    model.to(device)  # Ensure the model is on the correct device
    catalog = pystac_client.Client.open(STAC_API)
    search = catalog.search(
        collections=[COLLECTION],
        datetime=f"{year}-01-01/{year}-12-31",
        bbox=(lon - 1e-5, lat - 1e-5, lon + 1e-5, lat + 1e-5),
        max_items=10,
        query={"eo:cloud_cover": {"lt": 80}},
    )

    all_items = search.get_all_items()
    items = list(all_items)
    if not items:
        return None
    
    items = sorted(items, key=lambda x: x.properties.get('eo:cloud_cover', float('inf')))
    lowest_cloud_item = items[0]

    epsg = lowest_cloud_item.properties["proj:epsg"]

    poidf = gpd.GeoDataFrame(
        pd.DataFrame(),
        crs="EPSG:4326",
        geometry=[Point(lon, lat)],
    ).to_crs(epsg)

    coords = poidf.iloc[0].geometry.coords[0]

    size = 256
    gsd = 30
    bounds = (
        coords[0] - (size * gsd) // 2,
        coords[1] - (size * gsd) // 2,
        coords[0] + (size * gsd) // 2,
        coords[1] + (size * gsd) // 2,
    )

    stack = stackstac.stack(
        lowest_cloud_item,
        bounds=bounds,
        snap_bounds=False,
        epsg=epsg,
        resolution=gsd,
        dtype="float32",
        rescale=False,
        fill_value=0,
        assets=["blue", "green", "red", "nir"],
        resampling=Resampling.nearest,
    )

    stack = stack.compute()

    items = []
    dates = []
    for item in all_items:
        if item.datetime.date() not in dates:
            items.append(item)
            dates.append(item.datetime.date())

    platform = "landsat-c2l1"
    mean = []
    std = []
    waves = []
    for band in stack.band:
        mean.append(metadata[platform].bands.mean[str(band.values)])
        std.append(metadata[platform].bands.std[str(band.values)])
        waves.append(metadata[platform].bands.wavelength[str(band.values)])

    transform = v2.Compose([v2.Normalize(mean=mean, std=std)])

    datetimes = stack.time.values.astype("datetime64[s]").tolist()
    times = [normalize_timestamp(dat) for dat in datetimes]
    week_norm = [dat[0] for dat in times]
    hour_norm = [dat[1] for dat in times]

    latlons = [normalize_latlon(lat, lon)] * len(times)
    lat_norm = [dat[0] for dat in latlons]
    lon_norm = [dat[1] for dat in latlons]

    pixels = torch.from_numpy(stack.data.astype(np.float32)).to(device)
    pixels = transform(pixels)

    batch_size = 16
    num_batches = math.ceil(len(stack) / batch_size)
    
    embeddings_list = []
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(stack))
        
        batch_pixels = pixels[start_idx:end_idx].to(device)
        batch_time = torch.tensor(np.hstack((week_norm, hour_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
        batch_latlon = torch.tensor(np.hstack((lat_norm, lon_norm))[start_idx:end_idx], dtype=torch.float32).to(device)
        
        batch_datacube = {
            "platform": platform,
            "time": batch_time,
            "latlon": batch_latlon,
            "pixels": batch_pixels,
            "gsd": torch.tensor(stack.gsd.values).to(device),
            "waves": torch.tensor(waves).to(device),
        }

        batch_datacube = to_device(batch_datacube, device)

        try:
            model = model.to(device)

            with torch.no_grad():
                unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
            batch_embeddings = unmsk_patch[:, 0, :].cpu().numpy()
            embeddings_list.append(batch_embeddings)
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"GPU OOM for point ({lon}, {lat}), batch {i+1}/{num_batches}. Trying CPU...")
                device = torch.device("cpu")
                batch_datacube = to_device(batch_datacube, device)
                model = model.to(device)
                with torch.no_grad():
                    unmsk_patch, _, _, _ = model.model.encoder(batch_datacube)
                batch_embeddings = unmsk_patch[:, 0, :].numpy()
                embeddings_list.append(batch_embeddings)
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            else:
                raise e

    embeddings = np.concatenate(embeddings_list, axis=0)
    return embeddings

# Specify the years for the datetime range in the search
year = 2013

# Initialize an empty dictionary to store results for both years
results_dict = {"lon": [], "lat": [], "embeddings": []}

# Iterate through the points and process each one for both years
for i, point in enumerate(tqdm(points)):
    lon, lat = point
    results_dict["lon"].append(lon)
    results_dict["lat"].append(lat)
    
    embeddings = process_point(lon, lat, model, metadata, year, device, i)
    if embeddings is not None:
        results_dict["embeddings"].append(embeddings)
    else:
        results_dict["embeddings"].append(None)

# Create a DataFrame from the results
df = pd.DataFrame(results_dict)

# Convert to a GeoDataFrame
gdf_results = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# Output the resulting GeoDataFrame
gdf_results.head()


100%|██████████| 238/238 [03:24<00:00,  1.16it/s]


,lon,lat,embeddings,geometry
0,108.539475,34.270343,"[[0.040251017, -0.09942115, -0.05468018, 0.221...",POINT (108.53948 34.27034)
1,108.567260,34.270898,"[[0.044807635, -0.11502899, -0.046713702, 0.22...",POINT (108.56726 34.27090)
2,108.595045,34.271447,"[[0.07913431, -0.104119375, -0.06128583, 0.237...",POINT (108.59504 34.27145)
3,108.622831,34.271989,"[[0.10027202, -0.097676344, -0.09782456, 0.220...",POINT (108.62283 34.27199)
4,108.650617,34.272525,"[[0.1512495, -0.06380439, -0.10349344, 0.18129...",POINT (108.65062 34.27253)


In [69]:
gdf_copy = gdf_results.copy()
gdf_copy["embeddings"] = [embedding.flatten() if embedding is not None and embedding.size > 0 else None for embedding in gdf_results["embeddings"]]

gdf_copy.to_parquet("test_data/challenge_4.parquet")

In [70]:
import numpy as np
import tensorflow as tf
import optuna
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

class NeuralNetwork:
    def __init__(self, input_shape, layers, dropout_rate, learning_rate, device):
        self.input_shape = input_shape
        self.layers = layers
        self.dropout_rate = dropout_rate
        self.learning_rate = learning_rate
        self.device = device
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Input(shape=(self.input_shape,)))
        for layer_size in self.layers:
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(self.dropout_rate))
        model.add(Dense(2, activation='linear'))  # Output layer for predicting 'maize' and 'wheat'
        return model

    def compile_model(self):
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])  # Use MSE for regression

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        with tf.device(self.device):
            history = self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        return history

    def evaluate_model(self, X_test, y_test):
        with tf.device(self.device):
            loss, mae = self.model.evaluate(X_test, y_test)
        return loss, mae

    def predict(self, X_test):
        with tf.device(self.device):
            predictions = self.model.predict(X_test)
        return predictions

    def calculate_rmse(self, y_test, predictions):
        return np.sqrt(mean_squared_error(y_test, predictions, multioutput='raw_values'))
    
    def save_model(self, filename):
        self.model.save(filename)

    @classmethod
    def load_model(cls, filename, input_shape, device):
        loaded_model = tf.keras.models.load_model(filename)
        nn = cls(input_shape, [], 0, 0, device)  # Dummy values for layers, dropout_rate, and learning_rate
        nn.model = loaded_model
        return nn
    
def objective(trial):
    layers = []
    for i in range(trial.suggest_int('n_layers', 1, 3)):
        layers.append(trial.suggest_int(f'n_units_l{i}', 64, 512))
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    
    nn = NeuralNetwork(input_shape=X_train.shape[1], device=device, layers=layers, dropout_rate=dropout_rate, learning_rate=learning_rate)
    nn.compile_model()
    
    nn.train_model(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
    
    predictions = nn.predict(X_test)
    rmse = nn.calculate_rmse(y_test, predictions)
    
    return np.mean(rmse)

# Assuming combined_gdf is already loaded
combined_gdf = gdf_copy.copy()
combined_gdf_filtered = combined_gdf.dropna(subset=['embeddings'])


In [71]:
# Detect if GPU is available
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

# Load the model
loaded_nn = NeuralNetwork.load_model('models/task_4_model.h5', input_shape=768, device=device)

# Prepare your new data (assuming it's in the same format as your training data)
new_data = np.squeeze(combined_gdf_filtered['embeddings'].tolist())
new_data = pd.DataFrame(new_data)  # Ensure the new data is in DataFrame format

# Make predictions
new_predictions = loaded_nn.predict(new_data)

gdf_copy[['pred_maize','pred_wheat']] = new_predictions

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 240ms/step


In [76]:
import geopandas as gpd
import rioxarray
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from shapely.geometry import Point

# Load the TIFF files as xarray Datasets
raster1_path = 'test_data/MaizeYield2013.tif'
raster2_path = 'test_data/WheatYield2013.tif'

# Convert the TIFFs to xarray datasets
raster1_xr = rioxarray.open_rasterio(raster1_path)
raster2_xr = rioxarray.open_rasterio(raster2_path)
raster1_xr = raster1_xr.rio.reproject(4326)
raster2_xr = raster2_xr.rio.reproject(4326)

# Create the geometry column using Point
gdf_copy['geometry'] = gdf_copy.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# Function to get raster value at geometry
def get_raster_value_at_geometry_xr(geometry, raster_xr):
    try:
        clipped = raster_xr.rio.clip([geometry], raster_xr.rio.crs)
        return float(clipped.sum().values)
    except rioxarray.exceptions.NoDataInBounds as e:
        return np.nan

def process_row(geom, raster_xr):
    return get_raster_value_at_geometry_xr(geom, raster_xr)

# Parallel processing with joblib
n_jobs = 12 # Use all available cores

# Use tqdm with joblib for progress bar
raster_values_1 = Parallel(n_jobs=n_jobs)(
    delayed(process_row)(geom, raster1_xr) for geom in tqdm(gdf_copy['geometry'], desc="Processing Raster 1"))

raster_values_2 = Parallel(n_jobs=n_jobs)(
    delayed(process_row)(geom, raster2_xr) for geom in tqdm(gdf_copy['geometry'], desc="Processing Raster 2"))

# Assign the values back to the GeoDataFrame
gdf_copy['maize'] = raster_values_1
gdf_copy['wheat'] = raster_values_2

# Set values greater than 1 to NaN for raster_value_1
gdf_copy.loc[np.abs(gdf_copy['maize']) > 1e37, 'maize'] = np.nan

# Set values greater than 1 to NaN for raster_value_2
gdf_copy.loc[np.abs(gdf_copy['wheat']) > 1e37, 'wheat'] = np.nan

gdf_copy['maize'] = gdf_copy['maize'].fillna(0)
gdf_copy['wheat'] = gdf_copy['wheat'].fillna(0)

# Save the result to a parquet file
gdf_copy.to_parquet("test_data/challenge_4.parquet")


Processing Raster 2: 100%|██████████| 238/238 [00:02<00:00, 98.71it/s]


In [77]:
print("Test set RMSE:", np.sqrt(np.mean((gdf_copy['pred_maize']-gdf_copy['maize'])**2)))
print("Test set corr:", np.corrcoef(gdf_copy['pred_maize'],gdf_copy['maize'])[0][1])

Test set RMSE: 3235.1679227391887
Test set corr: -0.12314035777962042


In [72]:
gdf_copy[['pred_maize','pred_wheat']] 

,pred_maize,pred_wheat
0,1000.960449,975.279114
1,996.647217,969.454590
2,969.085144,943.091736
3,817.738770,796.544983
4,550.917603,542.767334
...,...,...
233,331.973877,320.300781
234,473.824341,455.954010
235,650.904785,626.890137
236,823.732727,789.586365
